# Remove instrument resonse

This is a script which takes the cut A02 moonquake traces and removes the instrument response. 

The instrument response is read in from IRIS metatdata client, as this can deal with the different responses at dofferent times 

However, due to the different shapes of the responses, a different water level is different for the two different types. They probably should not be mixed so they are given different file ends. 

The input is the '.cut' files and the output is a mseed file with the appended name.  

In [1]:
from obspy import read,read_inventory
import glob
import os
import obspy.signal
from obspy.core import stream
from obspy.signal import filter
from  obspy.signal.cross_correlation import correlate
from  obspy.signal.cross_correlation import xcorr_max
from scipy import stats
from obspy.io.xseed import Parser
import datetime
from  obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

# Edit client to use your data center of interest
client = Client("IRIS")


In [2]:
year = ['1969','1970','1971','1972','1973','1974','1975','1976','1977']
station = ['S14','S15','S16']
comp = ['MH1','MH2','MHZ']

In [3]:
#defining the regions where the instrument response is flat. There are not many events due to the reduced sensitivity 
#And 
S12_flat_dates_start1 = UTCDateTime('1974-10-16T14:02:36.073')
S12_flat_dates_end1= UTCDateTime('1975-04-09T15:31:03.702')
S12_flat_dates_start2 = UTCDateTime('1975-06-28T13:48:23.124')
S12_flat_dates_end2 = UTCDateTime('1977-03-27T15:41:06.247')

S14_flat_dates_start = UTCDateTime('1976-09-18T08:24:35.026')
S14_flat_dates_end = UTCDateTime('1976-11-17T15:34:34.524')

S15_flat_dates_start1 = UTCDateTime('1971-10-24T20:58:47.248')
S15_flat_dates_end1= UTCDateTime('1971-11-08T00:34:39.747')
S15_flat_dates_start2 = UTCDateTime('1975-06-28T14:36:33.034')
S15_flat_dates_end2 = UTCDateTime('1977-03-27T15:24:05.361')

S16_flat_dates_start1 = UTCDateTime('1972-05-13T14:08:03.157')
S16_flat_dates_end1= UTCDateTime('1972-05-14T14:47:08.185')
S16_flat_dates_start2 = UTCDateTime('1975-06-29T02:46:45.610')
S16_flat_dates_end2 = UTCDateTime('1977-03-26T14:52:05.483')


In [ ]:

    for y in year:
        for sta in station:
            if sta == 'S14' or sta == 'S15':
                comp = ['MHN','MHE','MHZ']
            else: 
                comp = ['MH1','MH2','MHZ']
            for co in comp:
                print(y,sta,co)
                st = []
                fil='/Users/TheStuffofAlice/Dropbox/Apollo/A01_moonquakes/'+str(y)+'/'+str(sta)+'/'+str(co)+'/*'+co+'*cut'
                fnames=glob.glob(fil)
                for fname in fnames: 
                    st = obspy.read(fname)
                    print(st)
                    tr = st[0]
                    tr.detrend('linear')
                    tr.detrend('demean')
                    if tr.stats.npts < 2:
                        continue
                    if sta =='S12' and  S12_flat_dates_start1 < tr.stats.starttime < S12_flat_dates_end1:
                        water_level = 600
                        file_name = '.resp_flat'
                    elif sta =='S12' and  S12_flat_dates_start2 < tr.stats.starttime < S12_flat_dates_end2:
                        water_level = 600
                        file_name = '.resp_flat'
                    else: 
                        water_level = 0.0001
                        file_name = '.resp_peaked'
                    if sta =='S14' and  S14_flat_dates_start < tr.stats.starttime < S14_flat_dates_end:
                        water_level = 600
                        file_name = '.resp_flat'
                    else: 
                        water_level = 0.0001
                        file_name = '.resp_peaked'
                    if sta =='S15' and  S15_flat_dates_start1 < tr.stats.starttime < S15_flat_dates_end1:
                        water_level = 600
                        file_name = '.resp_flat'
                    elif sta =='S15' and  S15_flat_dates_start2 < tr.stats.starttime < S15_flat_dates_end2:
                        water_level = 600
                        file_name = '.resp_flat'
                    else: 
                        water_level = 0.0001
                        file_name = '.resp_peaked'
                    if sta =='S16' and  S16_flat_dates_start1 < tr.stats.starttime < S16_flat_dates_end1:
                        water_level = 600
                        file_name = '.resp_flat'
                    elif sta =='S16' and  S16_flat_dates_start2 < tr.stats.starttime < S16_flat_dates_end2:
                        water_level = 600
                        file_name = '.resp_flat'
                    else: 
                        water_level = 0.0001
                        file_name = '.resp_peaked'
                    t1 = tr.stats.starttime 
                    inv = client.get_stations(network="XA", station=sta, channel=co, level="response", starttime=t1)
                    st_copy=tr.copy()
                    tr.remove_response(inventory=inv,output="DISP",pre_filt = [0.01, 0.02, 10, 20],taper=5,water_level = 0.0001, zero_mean=True )
                    print('response removed!')
                    tr.write(fname.split('cut')[0]+str(file_name),format="MSEED")
                    print('file saved!')

1969 S14 MHN
1969 S14 MHE
1969 S14 MHZ
1969 S15 MHN
1969 S15 MHE
1969 S15 MHZ
1969 S16 MH1
1969 S16 MH2
1969 S16 MHZ
1970 S14 MHN
1970 S14 MHE
1970 S14 MHZ
1970 S15 MHN
1970 S15 MHE
1970 S15 MHZ
1970 S16 MH1
1970 S16 MH2
1970 S16 MHZ
1971 S14 MHN
1 Trace(s) in Stream:
XA.S14..MHN | 1971-12-03T15:25:59.953057Z - 1971-12-03T23:59:59.953057Z | 6.6 Hz, 204316 samples


/Users/TheStuffofAlice/opt/anaconda2/envs/roses/lib/python3.7/site-packages/obspy/io/stationxml/core.py:84: UserWarning: The StationXML file has version 1.1, ObsPy can deal with version 1.0. Proceed with caution.
  root.attrib["schemaVersion"], SCHEMA_VERSION))
/Users/TheStuffofAlice/opt/anaconda2/envs/roses/lib/python3.7/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHN | 1971-04-18T06:38:59.957428Z - 1971-04-19T00:00:00.032900Z | 6.6 Hz, 413799 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHN | 1971-11-04T17:44:59.969528Z - 1971-11-04T23:59:59.894056Z | 6.6 Hz, 149063 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHN | 1971-09-02T22:42:00.056000Z - 1971-09-03T00:00:00.056000Z | 6.6 Hz, 31006 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHN | 1971-09-01T05:45:00.062428Z - 1971-09-01T23:59:59.836013Z | 6.6 Hz, 435262 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHN | 1971-11-23T04:50:00.011900Z - 1971-11-23T06:20:00.011900Z | 6.6 Hz, 35776 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHN | 1971-08-08T10:10:00.016000Z - 1971-08-08T23:59:59.865057Z | 6.6 Hz, 329925 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHN | 1971-12-21T00:22:59.960428Z - 1971-12-21T0

response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHE | 1971-09-06T18:41:59.941957Z - 1971-09-06T19:59:59.941957Z | 6.6 Hz, 31006 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHE | 1971-11-25T08:48:00.048000Z - 1971-11-25T10:24:59.972528Z | 6.6 Hz, 38558 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHE | 1971-10-28T02:52:59.997428Z - 1971-10-28T23:59:59.921956Z | 6.6 Hz, 503633 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHE | 1971-11-24T05:35:00.025428Z - 1971-11-24T07:09:59.949956Z | 6.6 Hz, 37763 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHE | 1971-10-29T01:12:00.015000Z - 1971-10-29T02:40:00.015000Z | 6.6 Hz, 34981 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHE | 1971-10-25T09:24:59.943528Z - 1971-10-25T10:44:59.943528Z | 6.6 Hz, 31801 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S14..MHE | 1971-09-30T20:02:59.953458Z - 1971-09-30T21:2

response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHN | 1971-09-01T05:45:00.031528Z - 1971-09-01T23:59:59.956056Z | 6.6 Hz, 435263 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHN | 1971-10-28T02:52:59.925528Z - 1971-10-29T00:00:00.001000Z | 6.6 Hz, 503634 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHN | 1971-11-25T08:48:00.026900Z - 1971-11-25T10:24:59.951428Z | 6.6 Hz, 38558 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHN | 1971-09-06T18:42:00.075000Z - 1971-09-06T20:00:00.075000Z | 6.6 Hz, 31006 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHN | 1971-11-24T05:35:00.060428Z - 1971-11-24T07:09:59.984956Z | 6.6 Hz, 37763 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHN | 1971-10-29T01:11:59.947957Z - 1971-10-29T02:39:59.947957Z | 6.6 Hz, 34981 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHN | 1971-09-30T20:03:00.041428Z - 1971-09-30T21:

response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHZ | 1971-11-25T08:48:00.026900Z - 1971-11-25T10:24:59.951428Z | 6.6 Hz, 38558 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHZ | 1971-11-24T05:35:00.060428Z - 1971-11-24T07:09:59.984956Z | 6.6 Hz, 37763 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHZ | 1971-10-29T01:11:59.947957Z - 1971-10-29T02:39:59.947957Z | 6.6 Hz, 34981 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHZ | 1971-09-30T20:03:00.041428Z - 1971-09-30T21:19:59.965956Z | 6.6 Hz, 30608 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHZ | 1971-10-25T09:25:00.035528Z - 1971-10-25T10:45:00.035528Z | 6.6 Hz, 31801 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHZ | 1971-10-26T22:43:00.010428Z - 1971-10-26T23:59:59.934956Z | 6.6 Hz, 30608 samples
response removed!
file saved!
1 Trace(s) in Stream:
XA.S15..MHZ | 1971-09-28T00:00:00.002000Z - 1971-09-28T01:00

In [6]:
st = read('/Users/TheStuffofAlice/Dropbox/Apollo/A01_moonquakes/1976/S14/MHN/S14.XA..MHN.1976.77.cut',format="MSEED")
st.plot()

ObsPyMSEEDFilesizeTooSmallError: The smallest possible mini-SEED record is made up of 128 bytes. The passed buffer or file contains only 0.